# Temperature Control

Import necessary packages

In [ ]:
from distutils.log import error
import board
import digitalio
import adafruit_max31856
import time
from datetime import datetime as dt
import csv
import matplotlib
matplotlib.use("tkAgg")
import matplotlib.pyplot as plt
import numpy as np
import keyboard
from tclab import clock, setup, Historian, Plotter
import PID
import os
from IPython.display import clear_output

Get current directory so we can save log files in a separate folder.

In [25]:
ROOT_DIR = os.path.realpath(os.path.join(os.path.dirname("Temperature Control.ipynb")))

Define the temperature and error logging functions that output to csv and text files respectivly.

In [26]:
def log_error(f_name,error,time,first):
    if first==True:
        with open(os.path.join(ROOT_DIR, 'Logs', f_name),'w',encoding='UTF8',newline='') as err_log_file:
            err_log_file.write('')
    else:
        with open(os.path.join(ROOT_DIR, 'Logs', f_name),'a',encoding='UTF8',newline='') as err_log_file:
            err_log_file.write("\n".join('{} at t= {}'.format(error,str(time))))

def log_temps(f_name,header,data,first):
    if first==True:
        with open(os.path.join(ROOT_DIR, 'Logs', f_name),'w',encoding='UTF8', newline='') as temp_log_file:
            temp_log_w=csv.writer(temp_log_file)    #temperature log file
            temp_log_w.writerow(header)
    else:
        with open(os.path.join(ROOT_DIR, 'Logs', f_name),'a',encoding='UTF8', newline='') as temp_log_file:
            temp_log_w=csv.writer(temp_log_file)    #temperature log file
            temp_log_w.writerow(data)

Create the temperature and error log files.

In [27]:
data_f_name=ROOT_DIR+'/Logs/Temp log {}.csv'.format(dt.now().strftime('%m-%d-%Y, %H-%M'))
data_header=['Rt', 'temp_hex', 'temp_ch' , 'temp_chamber', 'MV','Heater On']
log_temps(data_f_name,data_header,[0,0,0,0,0,0],True)
updated=False

Define the coldhead, heat exchanger, chamber, and heater entities that will be controled.

In [28]:
# Create sensor object, communicating over the board's default SPI bus
spi = board.SPI()

# allocate a CS pin and set the direction
cs13 = digitalio.DigitalInOut(board.D13)
cs13.direction = digitalio.Direction.OUTPUT
cs25 = digitalio.DigitalInOut(board.D25)
cs25.direction = digitalio.Direction.OUTPUT
cs26 = digitalio.DigitalInOut(board.D26)
cs26.direction = digitalio.Direction.OUTPUT
Heater = digitalio.DigitalInOut(board.D17)
Heater.direction= digitalio.Direction.OUTPUT

# create a thermocouple object with the above
HeatEx = adafruit_max31856.MAX31856(spi, cs26,thermocouple_type=adafruit_max31856.ThermocoupleType.T)
ColdHead = adafruit_max31856.MAX31856(spi, cs13,thermocouple_type=adafruit_max31856.ThermocoupleType.T)
Chamber = adafruit_max31856.MAX31856(spi, cs25,thermocouple_type=adafruit_max31856.ThermocoupleType.T)


Initiate the PID temperature control.

In [29]:
Heater.value = False
Heat_on=True
targetT = -135    #initial inputs for PID control
P = 10
I = 1
D = 1

controller = PID.PID(P, I, D)        # create pid control
controller.SetPoint = targetT              # initialize
controller.setSampleTime(1)

Read the coldhead, heat exhanger and chamber temperatures then log them.

In [30]:
temp_coldhead=ColdHead.temperature
temp_heatex=HeatEx.temperature
temp_chamber=Chamber.temperature

log_temps(data_f_name,data_header,[dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_heatex, temp_coldhead, temp_chamber, controller.output,"False"],False)

Create the temperature plot

In [31]:
plot_window = 10
coldhead_temps = np.array(np.zeros([plot_window]))
heatex_temps = np.array(np.zeros([plot_window]))
chamber_temps = np.array(np.zeros([plot_window]))
time_stamps = []
#np.array(np.zeros([plot_window]))
for i in range(plot_window):
    time_stamps.append(dt.now().strftime('%H:%M:%S'))
    #x_var.append(time.asctime(time.time()))
plt.ion()
fig, ax = plt.subplots()
chline,  = ax.plot(time_stamps, coldhead_temps, label='Cold Head')
hexline, = ax.plot(time_stamps, heatex_temps, label='Heat Exchanger')
chamberline, =ax.plot(time_stamps, chamber_temps, label='Chamber')
plot_window = 3000
ax.locator_params(tight=True, nbins=4)
ax.legend()
ax.set_xlabel('Time (Hour:Min)')
ax.set_ylabel('Temperature (Celsius)')

/tmp/ipykernel_2479/1718940601.py:16: UserWarning: 'set_params()' not defined for locator of type <class 'matplotlib.category.StrCategoryLocator'>
  ax.locator_params(tight=True, nbins=4)


Text(0, 0.5, 'Temperature (Celsius)')

Loop that reads the temperatures of the coldhead, heat exhanger and chamber logs them, updates the PID control and runs the heater, then plots the tmperatures.

In [ ]:
print_log=[]
while True:
    clear_output(wait=True)
    if os.stat(data_f_name).st_size>= 4,194,304:
            data_f_name=ROOT_DIR+'/Logs/Temp log {}.csv'.format(dt.now().strftime('%m-%d-%Y, %H-%M'))
            updated=True
    temp_coldhead=ColdHead.temperature
    temp_heatex=HeatEx.temperature
    temp_chamber=Chamber.temperature
    controller.update(temp_heatex) # compute manipulated variable
    MV = controller.output # apply
    if MV > 0:
       Heater.value = True
       Heat_on=True
       # write to event log that heater turned on/off
    else:
       Heater.value = False
       Heat_on=False
    if updated:
        if Heat_on:
            log_temps(data_f_name,data_header,[0,0,0,0,0,0],True)
            log_temps(data_f_name,data_header,[dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_heatex, temp_coldhead, temp_chamber, MV,'On'],False)   #push the temperature readings to the log file
            updated=False
        else:
            log_temps(data_f_name,data_header,[0,0,0,0,0,0],True)
            log_temps(data_f_name,data_header,[dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_heatex, temp_coldhead, temp_chamber, MV,'Off'],False)
            updated=False
    else:
        if Heat_on:
            log_temps(data_f_name,data_header,[dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_heatex, temp_coldhead, temp_chamber, MV,'On'],False)   #push the temperature readings to the log file
        else:
            log_temps(data_f_name,data_header,[dt.now().strftime('%Y-%m-%d %H:%M:%S'), temp_heatex, temp_coldhead, temp_chamber, MV,'Off'],False)
    if len(print_log)<=20:
        if Heat_on:
            print_log.append(dt.now().strftime('%Y-%m-%d %H:%M:%S'),temp_coldhead, temp_heatex, temp_chamber, MV,"On")
        else:
            print_log.append(dt.now().strftime('%Y-%m-%d %H:%M:%S'),temp_coldhead, temp_heatex, temp_chamber, MV,"Off")
    else:
        print_log.pop(0)
        if Heat_on:
            print_log.append(dt.now().strftime('%Y-%m-%d %H:%M:%S'),temp_coldhead, temp_heatex, temp_chamber, MV,"On")
        else:
            print_log.append(dt.now().strftime('%Y-%m-%d %H:%M:%S'),temp_coldhead, temp_heatex, temp_chamber, MV,"Off")
    for line in print_log:
        print(line)
    coldhead_temps = np.append(coldhead_temps, temp_coldhead)
    heatex_temps = np.append(heatex_temps, temp_heatex)
    chamber_temps = np.append(chamber_temps, temp_chamber)
    time_stamps.append(dt.now().strftime('%H:%M:%S'))
    coldhead_temps = coldhead_temps[1: plot_window + 1]
    heatex_temps = heatex_temps[1:plot_window+1]
    chamber_temps = chamber_temps[1:plot_window+1]
    time_stamps = time_stamps[1: plot_window + 1]
    chline.set_ydata(coldhead_temps)
    chline.set_xdata(time_stamps)
    hexline.set_ydata(heatex_temps)
    hexline.set_xdata(time_stamps)
    chamberline.set_ydata(chamber_temps)
    chamberline.set_xdata(time_stamps)
    ax.relim()
    #ax.set_ylim(0,24)
    ax.autoscale_view()
    ax.set_xticks(time_stamps[::100])
    ax.set_xticklabels(time_stamps[::100])
    fig.canvas.draw()
    fig.canvas.flush_events()
    time.sleep(10)

2022-09-22 12:57:09 -116.640625 -178.2890625 -124.03125 137.99225906452943 On
2022-09-22 12:57:11 -116.6796875 -178.15625 -123.953125 136.82127452070668 On
2022-09-22 12:57:13 -116.640625 -178.2890625 -123.9140625 136.38283524427362 On
2022-09-22 12:57:14 -116.546875 -178.1796875 -123.875 135.41074120225866 On
2022-09-22 12:57:16 -116.5859375 -178.2578125 -123.8203125 135.88310700563113 On
2022-09-22 12:57:17 -116.6640625 -178.4140625 -123.8984375 136.6882705455239 On
2022-09-22 12:57:19 -116.5859375 -178.3671875 -123.78125 135.81443272081967 On
2022-09-22 12:57:21 -116.703125 -178.203125 -123.8046875 137.10561868998263 On
2022-09-22 12:57:22 -116.5703125 -178.546875 -123.9140625 135.62638372682358 On
2022-09-22 12:57:24 -116.5859375 -178.4140625 -123.8046875 135.86888431052276 On
2022-09-22 12:57:26 -116.6796875 -178.2578125 -123.8046875 136.8537787816755 On
2022-09-22 12:57:27 0.0 -178.2578125 -123.765625 -1143.330037859984 Off
2022-09-22 12:57:29 -108.953125 -178.15625 -123.8203125 